In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import AlgoLoop
import StatisticFunc

plt.style.use('seaborn-deep')                        # 绘图风格
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 字体雅黑
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False    # 处理负号

In [ ]:
print(plt.style.available)

# 预处理

## 提取并处理数据

### 各种利率

In [ ]:
# GC007 利率
GCRates = pd.read_excel(io="Raw/GC007利率.xlsx")              
GCRates = GCRates.drop([0, 1, 2], axis=0)                    
GCRates.columns = ['日期', 'GC007']                           
GCRates["日期"] = pd.DatetimeIndex(GCRates["日期"])           
GCRates = GCRates.set_index("日期")                          

In [ ]:
# 10年国债收益率
CNRates = pd.read_excel(io="Raw/10年国债到期收益率.xls")        
CNRates.columns = ['日期', '十年国债收益率']                    
CNRates = CNRates.drop([0, 3870, 3871], axis=0)              
CNRates["日期"] = pd.DatetimeIndex(CNRates["日期"])           
CNRates = CNRates.set_index("日期")                          

In [ ]:
# 10年美债收益率
USRates = pd.read_excel(io="Raw/10年美债到期收益率.xlsx")                
USRates.columns = ['日期', '十年美债收益率']                             
USRates = USRates.drop([0, 6263, 6264], axis=0)                       
USRates["日期"] = pd.DatetimeIndex(USRates["日期"])                    
USRates = USRates.set_index("日期")                                   

In [ ]:
# 合并利率
Rates = GCRates.merge(CNRates, how='left', left_index=True, right_index=True)
Rates = Rates.merge(USRates, how='left', left_index=True, right_index=True)

### 权益类资产换手率

In [ ]:
Turnovers = pd.read_excel(io="Raw/资产换手率.xlsx")                         
Turnovers = Turnovers.drop([0], axis=0)                                   
Turnovers["日期"] = pd.DatetimeIndex(Turnovers["日期"])                    
Turnovers = Turnovers.set_index("日期")                                   

In [ ]:
# 时间对齐
Turnovers = Turnovers.loc[Rates.index, :]

### 铁矿石价格指数

In [ ]:
IronStone = pd.read_excel(io="Raw/铁矿石价格指数.xls")                      
IronStone = IronStone.drop([0, 2642, 2643], axis=0)                       
IronStone.columns=['日期', '铁矿石期货']                                    
IronStone["日期"] = pd.DatetimeIndex(IronStone["日期"])                    
IronStone = IronStone.set_index("日期")                                   

### 各资产收盘价

In [ ]:
Assets = pd.read_excel(io="Raw/资产收盘价.xlsx")                      
Assets = Assets.drop([0], axis=0)                                   
Assets["日期"] = pd.DatetimeIndex(Assets["日期"])                    
Assets = Assets.set_index("日期")                                   
Assets = Assets.loc[Rates.index, :]                                

In [ ]:
# 整合铁矿石数据
Assets = Assets.drop('中信证券铁矿石', axis=1)
Assets = Assets.merge(IronStone, how='left', left_index=True, right_index=True)

# 填充铁矿石2011年前缺失数据
Assets['铁矿石期货'] = Assets['铁矿石期货'].fillna(method='bfill')

### 十年国债&美债价格指数

In [ ]:
# 计算持债时间
HoldPeriod = (Assets.index - Assets.index[0]).days

# 计算十年国债
Assets['10年国债'] = 100 - (Rates['十年国债收益率'] - 3.0) * 8.2 + 3.0 * HoldPeriod / 365.0
Assets['10年国债'] = Assets['10年国债'].fillna(method='ffill')

#计算十年美债
Assets['10年美债'] = 100 - (Rates['十年美债收益率'] - 3.0) * 8.2 + 3.0 * HoldPeriod / 365.0
Assets['10年美债'] = Assets['10年美债'].fillna(method='ffill')

In [ ]:
# 替换上证十年国债指数
Assets = Assets.drop('上证10年国债', axis=1)

In [ ]:
Assets.head()

### 美元汇率

In [ ]:
FXRates = pd.read_excel(io="Raw/美元汇率.xlsx")
FXRates = FXRates.drop([0, 1, 2], axis=0)
FXRates.columns = ['日期', '美元指数', '美元汇率']                        
FXRates['日期'] = pd.DatetimeIndex(FXRates['日期'])
FXRates = FXRates.set_index('日期')

In [ ]:
# # 皮尔逊相关系数
# np.corrcoef( FXRates['美元汇率'].values.flatten().astype(np.float32)[900:-233], 
#              (Assets['10年国债'] - Assets['10年美债']).values.astype(np.float32)[900:-220] )

In [ ]:
FXRates

## 计算各资产收益率

### 日内损益

In [ ]:
Returns = Assets.pct_change(axis=0)
Returns = Returns.dropna(axis=0, how='all')           # 删除无数据日

### 累计损益

In [ ]:
cumReturns = (1.0 + Returns)
cumReturns = cumReturns.fillna(1.0)                    # 填充空值
cumReturns = cumReturns.cumprod()                      # 计算各资产累计收益率

### 杠杆调整后的债券净值

In [ ]:
lever = 2.0                                 # 杠杆（额外）
leverReturns = Returns.copy()               # 创建副本 
leverCumReturns = cumReturns.copy()   

In [ ]:
# 更新杠杆调整后的债券净值
leverReturns['10年国债'] = Returns['10年国债'] * (1.0 + lever) - Rates.loc[Returns.index, 'GC007'] * lever / (365.0 * 100)
leverReturns['信用债3-5AAA'] = Returns['信用债3-5AAA'] * (1.0 + lever) - Rates.loc[Returns.index, 'GC007'] * lever / (365.0 * 100)

# 更新累计净值
leverCumReturns = (1.0 + leverReturns)
leverCumReturns = leverCumReturns.fillna(1.0)                    # 填充空值
leverCumReturns = leverCumReturns.cumprod()                      # 计算各资产累计收益率

### 剔除美元资产

In [ ]:
leverReturns_Domestic    = leverReturns.drop(['标普500', '10年美债'], axis=1)
leverCumReturns_Domestic = leverCumReturns.drop(['标普500', '10年美债'], axis=1)

In [ ]:
leverCumReturns_Domestic.plot(figsize=(16, 6))

# 风险平价 （杠杆+指数平均）

## 程序运行

In [ ]:
tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns_Domestic, leverCumReturns_Domestic, 
                                       Turnovers, FXRates['美元汇率'], mode='ema')

In [ ]:
StatisticFunc.WeightPlot(tradeDF, weightDF, '基准模型')

## 回测表现汇总

In [ ]:
smryDF = StatisticFunc.summaryDF(tradeDF)
pfmcDF = StatisticFunc.performanceDF(smryDF, tradeDF, name='基准模型')

In [ ]:
smryDF

In [ ]:
pfmcDF

## 收益率贡献度

In [ ]:
annualContrb = StatisticFunc.AnnualContribution(tradeDF)

In [ ]:
StatisticFunc.BarPlot(annualContrb, '基准模型')

## 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 6))

plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, label='基准模型')
    
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['标普500'], label='标普500')
plt.plot(cumReturns.index, cumReturns['10年国债'], label='10年国债')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均）')

plt.savefig('Pics/净值曲线（杠杆，指数平均）.png')

# 流动性与投资组合表现的关系

In [ ]:
# 投资组合日收益率
PnL = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1, axis=0) - 1.0
PnL = PnL.dropna()

## 二级市场流动性

- A股成交量、换手率
- 美股成交量、换手率

In [ ]:
# 基本处理与拆分数据
StockData = pd.read_excel(io="Raw/二级市场流动性.xlsx")
StockData.columns = StockData.iloc[1, :]

Volume = StockData.iloc[:, :4]
Volume = Volume.drop(index=[0, 1, 2])
Volume = Volume.set_index(['日期'])

Turnover = StockData.iloc[:, 5:]
Turnover = Turnover.drop(index=[0, 1, 2])
Turnover = Turnover.set_index(['日期'])

In [ ]:
# 成交量相关系数
data1 = Volume.loc[PnL.index, '沪深300'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 成交量
fig, ax1 = plt.subplots(1, 1, figsize=(16,6), dpi=150)

ax1.plot(Volume.index, Volume['沪深300'].ewm(120).mean(), label='沪深300（20移动平均）', zorder=0)
ax1.plot(Volume.index, Volume['中证500'].ewm(120).mean(), label='中证500（20移动平均）', zorder=1)

ax1.legend(loc='upper left')
ax1.set_xlabel('时间')
ax1.set_ylabel('成交量')
ax1.grid(True)

ax2 = ax1.twinx()  
# ax2.plot(tradeDF.index, tradeDF['投资组合净值']/10000, label='模型A_净值', color='tab:red')
ax2.bar(PnL.index, PnL.ewm(120).mean(), label='模型A_日收益率（20移动平均）', color='tab:orange')

ax2.legend(loc='upper right')
ax2.set_ylabel('投资组合日收益率')

plt.title('成交量变化与投资组合收益率的关系')
plt.show()

In [ ]:
# 换手率相关系数
data1 = Turnover.loc[PnL.index, '沪深300'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 换手率
fig, ax1 = plt.subplots(1, 1, figsize=(16,6), dpi=100)

ax1.plot(Turnover.index, Turnover['沪深300'].ewm(120).mean(), label='沪深300（20移动平均）', zorder=1)
ax1.plot(Turnover.index, Turnover['中证500'].ewm(120).mean(), label='中证500（20移动平均）', zorder=0)

ax1.legend(loc='upper left')
ax1.set_xlabel('时间')
ax1.set_ylabel('换手率')
ax1.grid(True)

ax2 = ax1.twinx()  
# ax2.plot(tradeDF.index, tradeDF['投资组合净值']/10000, label='模型A_净值', color='tab:red')
ax2.bar(PnL.index, PnL.ewm(120).mean(), label='模型A_日收益率（20移动平均）', color='tab:orange')

ax2.legend(loc='upper right')
ax2.set_ylabel('投资组合日收益率')

plt.title('换手率变化与投资组合收益率的关系')
plt.show()

## 广义信贷市场流动性

- 社会融资规模累计同比 - m2同比
- m2同比 - 名义GDP

In [ ]:
# 基本处理
CredData = pd.read_excel(io='Raw/广义信贷市场流动性.xls')
CredData = CredData.drop([0, 3841, 3842], axis=0)
CredData.columns = ['日期', 'M2同比 - GDP同比', '10年美债收益率', 'M2同比', 'M1同比', 'M1同比 - M2同比', '社会融资规模同比 - M2同比']
CredData = CredData.set_index('日期')

# 用季度数据填充日数据
CredData = CredData.fillna(method='bfill', axis=0)

In [ ]:
# M2同比 - GDP同比 相关系数
data1 = CredData.loc[PnL.index, 'M2同比 - GDP同比'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 社融规模 - M2同比 相关系数
data1 = CredData.loc[PnL.index, '社会融资规模同比 - M2同比'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 广义信贷市场流动性
fig, ax1 = plt.subplots(1,1,figsize=(16,6), dpi=150)

ax1.plot(CredData.index, CredData['M2同比 - GDP同比'].ewm(120).mean(), 
         label='M2同比 - GDP同比（移动平均120）', zorder=0)
ax1.plot(CredData.index, CredData['社会融资规模同比 - M2同比'].ewm(120).mean(), 
         label='社会融资规模同比 - M2同比（移动平均120）', zorder=1)

ax1.legend(loc='upper left')
ax1.set_xlabel('时间')
ax1.set_ylabel('社会融资规模同比 - M2同比')
ax1.grid(True)

ax2 = ax1.twinx()  
# ax2.plot(tradeDF.index, tradeDF['投资组合净值']/10000, label='净值(模型A)', color='tab:red')
ax2.bar(PnL.index, PnL.ewm(120).mean(), label='模型A_日收益率（120移动平均）', color='tab:orange')

ax2.legend(loc='upper right')
ax2.set_ylabel('投资组合净值')

plt.title('社会融资规模同比 - M2同比变化与投资组合净值')
plt.show()

## 狭义金融市场流动性（利率）

- 超储率：超额存款准备金占存款的比例称为超额存款准备金率

- 10年国债收益率

- 10年美债收益率

- 美元指数

In [ ]:
# 基本处理
FinData = pd.read_excel(io='Raw/狭义金融市场流动性.xlsx')
FinData.columns = FinData.iloc[1, :]
FinData = FinData.drop([0, 1, 2], axis=0)
FinData = FinData.set_index('日期')

# 填充日数据
FinData = FinData.fillna(method='bfill', axis=0)

In [ ]:
# 超储率相关系数
data1 = FinData.loc[PnL.index, '超额存款准备金率(超储率):金融机构'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 10年国债收益率
data1 = Returns.loc[PnL.index, '10年国债'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 10年美债收益率
data1 = Returns.loc[PnL.index, '10年美债'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 美元价格指数
data1 = FXRates.loc[PnL.index, '美元指数'].ewm(120).mean()
data2 = PnL.ewm(120).mean()

np.corrcoef(data1, data2)

In [ ]:
# 超储率
fig, ax1 = plt.subplots(1,1,figsize=(16,6), dpi=150)

ax1.plot(FinData.index, FinData['超额存款准备金率(超储率):金融机构'].ewm(120).mean(), label='超储率（120移动平均）', zorder=0)


ax1.legend(loc='upper left')
ax1.set_xlabel('时间')
ax1.set_ylabel('比率')
ax1.grid(True)

ax2 = ax1.twinx()  
ax2.bar(PnL.index, PnL.ewm(120).mean(), label='模型A_日收益率（120移动平均）', color='tab:orange')

ax2.legend(loc='upper right')
ax2.set_ylabel('投资组合净值')

plt.title('狭义金融市场流动性与投资组合收益率变化')
plt.show()

In [ ]:
# 10年国债
fig, ax1 = plt.subplots(1,1,figsize=(16,6), dpi=150)

ax1.plot(Returns.index, Returns['10年国债'].ewm(120).mean(), label='10年国债_日收益率（120移动平均）', zorder=0)

ax1.legend(loc='upper left')
ax1.set_xlabel('时间')
ax1.set_ylabel('比率')
ax1.grid(True)

ax2 = ax1.twinx()  
ax2.bar(PnL.index, PnL.ewm(120).mean(), label='模型A_日收益率（120移动平均）', color='tab:orange')

ax2.legend(loc='upper right')
ax2.set_ylabel('投资组合净值')

plt.title('狭义金融市场流动性与投资组合收益率变化')
plt.show()

In [ ]:
# 10年美债
fig, ax1 = plt.subplots(1,1,figsize=(16,6), dpi=150)

ax1.plot(Returns.index, Returns['10年美债'].ewm(120).mean(), label='10年美债_日收益率（120移动平均）', zorder=1)

ax1.legend(loc='upper left')
ax1.set_xlabel('时间')
ax1.set_ylabel('比率')
ax1.grid(True)

ax2 = ax1.twinx()  
ax2.bar(PnL.index, PnL.ewm(120).mean(), label='模型A_日收益率（120移动平均）', color='tab:orange')

ax2.legend(loc='upper right')
ax2.set_ylabel('投资组合净值')

plt.title('狭义金融市场流动性与投资组合收益率变化')
plt.show()

In [ ]:
# 美元价格指数
fig, ax1 = plt.subplots(1,1,figsize=(16,6), dpi=150)

ax1.plot(FXRates.index, FXRates['美元指数'].ewm(120).mean(), label='美元指数（120移动平均）', zorder=0)

ax1.legend(loc='upper left')
ax1.set_xlabel('时间')
ax1.set_ylabel('比率')
ax1.grid(True)

ax2 = ax1.twinx()  
ax2.bar(PnL.index, PnL.ewm(120).mean(), label='模型A_日收益率（120移动平均）', color='tab:orange')

ax2.legend(loc='upper right')
ax2.set_ylabel('投资组合净值')

plt.title('狭义金融市场流动性与投资组合收益率变化')
plt.show()